# 🚀 JetX Model Trainer - Interactive Colab Interface

Bu notebook, JetX tahmin modellerini etkileşimli bir arayüz ile eğitmenizi sağlar.

## ✨ Özellikler
- 🎯 **Model Seçimi**: Dropdown ile istediğiniz modeli seçin
- ⚙️ **Parametre Ayarlama**: Slider'lar ile parametreleri optimize edin
- 📊 **Real-time Progress**: Eğitim sürecini canlı takip edin
- 📈 **Model Karşılaştırma**: Eğitilmiş modellerin performanslarını karşılaştırın
- 💾 **Otomatik Kaydetme**: Modeller otomatik olarak kaydedilir

---

**⚠️ Önce Runtime > Change runtime type > GPU seçin!**

In [ ]:
# 🛠️ Sistem Kurulumu
import sys
import os
import subprocess
import importlib.util

def install_package(package):
    """Paketi yükle"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

def check_and_install(package, import_name=None):
    """Paketi kontrol et ve yoksa yükle"""
    if import_name is None:
        import_name = package
    
    spec = importlib.util.find_spec(import_name)
    if spec is None:
        print(f"📦 {package} yükleniyor...")
        install_package(package)
    else:
        print(f"✅ {package} zaten yüklü")

# Gerekli paketleri kontrol et ve yükle
required_packages = [
    ("ipywidgets", "ipywidgets"),
    ("tqdm", "tqdm"),
    ("plotly", "plotly"),
    ("seaborn", "seaborn")
]

for package, import_name in required_packages:
    check_and_install(package, import_name)

print("🎉 Kurulum tamamlandı!")

In [ ]:
# 📚 Import'lar ve Temel Kurulum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from tqdm.notebook import tqdm
import time
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# PyTorch ve TensorFlow import'ları
try:
    import torch
    import torch.nn as nn
    TORCH_AVAILABLE = True
    print(f"✅ PyTorch {torch.__version__} yüklü")
    if torch.cuda.is_available():
        print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
except ImportError:
    TORCH_AVAILABLE = False
    print("⚠️ PyTorch bulunamadı")

try:
    import tensorflow as tf
    TF_AVAILABLE = True
    print(f"✅ TensorFlow {tf.__version__} yüklü")
    gpu_devices = tf.config.experimental.list_physical_devices('GPU')
    if gpu_devices:
        print(f"🎯 GPU: {len(gpu_devices)} device(s) found")
except ImportError:
    TF_AVAILABLE = False
    print("⚠️ TensorFlow bulunamadı")

# Stil ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Temel kurulum tamamlandı!")

In [ ]:
# 📥 Repository Kurulumu
import os
import subprocess

def clone_or_update_repo():
    """Repository'yi klonla veya güncelle"""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"
    
    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True)
        os.chdir(project_dir)
    
    # Python path'e ekle
    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")
    
    return project_dir

# Repository'yi hazırla
try:
    PROJECT_DIR = clone_or_update_repo()
    print(f"📂 Proje dizini: {PROJECT_DIR}")
    
    # Models dizinini oluştur
    models_dir = "/content/trained_models"
    os.makedirs(models_dir, exist_ok=True)
    print(f"💾 Model dizini hazır: {models_dir}")
    
    print("🎉 Repository kurulumu tamamlandı!")
    
except Exception as e:
    print(f"❌ Repository kurulum hatası: {e}")
    print("🔧 Manuel kurulum gerekli olabilir")

In [ ]:
# 📊 Gerçek JetX Veri Yükleme ve Rolling Window Sistemi
def load_real_jetx_data():
    """Gerçek JetX verilerini yükle"""
    try:
        # Önce repository'den veri yüklemeyi deneyelim
        from data_processing.loader import load_data_from_sqlite
        
        # Repository'deki veri dosyası
        repo_db_path = "/content/predictor_1/data/jetx_data.db"
        
        if os.path.exists(repo_db_path):
            data = load_data_from_sqlite(repo_db_path)
            if data and len(data) > 1000:
                print(f"✅ Repository'den {len(data)} kayıt yüklendi")
                return data
        
        print("⚠️ Repository'de yeterli veri bulunamadı, örnek veri oluşturuluyor...")
        return create_sample_jetx_data(6000)  # 6000 kayıt ile test
        
    except Exception as e:
        print(f"❌ Veri yükleme hatası: {e}")
        return create_sample_jetx_data(6000)

def create_sample_jetx_data(n_samples=6000):
    """Örnek JetX verisi oluştur (test için)"""
    np.random.seed(42)
    data = []
    
    for _ in range(n_samples):
        rand = np.random.random()
        if rand < 0.25:  # %25 crash
            value = np.random.uniform(1.0, 1.49)
        elif rand < 0.45:  # %20 düşük
            value = np.random.uniform(1.5, 2.5)
        elif rand < 0.70:  # %25 orta
            value = np.random.uniform(2.5, 5.0)
        elif rand < 0.90:  # %20 yüksek
            value = np.random.uniform(5.0, 20.0)
        else:  # %10 çok yüksek
            value = np.random.exponential(10.0) + 20.0
        
        data.append(round(value, 2))
    
    return data

def create_rolling_chunks(data, chunk_size=1000):
    """Veriyi rolling window için chunk'lara böl"""
    chunks = []
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        if len(chunk) >= chunk_size:  # Sadece tam chunk'ları al
            chunks.append(chunk)
    return chunks

# Gerçek veri yükleme
print("📊 Gerçek JetX verisi yükleniyor...")
jetx_data = load_real_jetx_data()

if jetx_data:
    # Veri yapısını kontrol et (tuple listesi mi, float listesi mi)
    is_tuple_list = isinstance(jetx_data[0], tuple)
    crash_values = [item[1] for item in jetx_data] if is_tuple_list else jetx_data

    print(f"📈 Veri istatistikleri:")
    print(f"   - Toplam kayıt: {len(crash_values)}")
    print(f"   - Min: {min(crash_values):.2f}")
    print(f"   - Max: {max(crash_values):.2f}")
    print(f"   - Ortalama: {np.mean(crash_values):.2f}")
    print(f"   - Crash oranı: {sum(1 for x in crash_values if x < 1.5)/len(crash_values)*100:.1f}%")
    
    # Rolling chunks oluştur
    rolling_chunks = create_rolling_chunks(jetx_data, 1000)
    print(f"📊 Rolling window chunks: {len(rolling_chunks)} adet (her biri 1000 kayıt)")
    
    # Global değişkenler
    JETX_DATA = jetx_data
    ROLLING_CHUNKS = rolling_chunks
    print("🎯 Gerçek veri hazırlama tamamlandı!")
else:
    print("❌ Veri hazırlama başarısız!")

In [ ]:
# 🏗️ Model Registry ve Yönetim Sistemi
class ModelRegistry:
    def __init__(self):
        self.trained_models = {}
        self.training_history = {}
        self.model_configs = {}
        self.model_paths = {}
        
    def register_model(self, model_name, model_type, config, performance_metrics):
        """Model'i kayıt et"""
        self.trained_models[model_name] = {
            'type': model_type,
            'config': config,
            'performance': performance_metrics,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'status': 'completed'
        }
        
    def get_model_summary(self):
        """Model özetini getir"""
        if not self.trained_models:
            return "Henüz eğitilmiş model yok."
        
        summary = "\n🎯 EĞİTİLMİŞ MODELLER:\n" + "="*50 + "\n"
        
        for name, info in self.trained_models.items():
            summary += f"📊 {name} ({info['type']})\n"
            summary += f"   ⏰ {info['timestamp']}\n"
            if 'performance' in info:
                perf = info['performance']
                summary += f"   📈 MAE: {perf.get('mae', 'N/A')}\n"
                summary += f"   📈 Accuracy: {perf.get('accuracy', 'N/A')}\n"
            summary += "\n"
        
        return summary
    
    def get_best_model(self, metric='mae'):
        """En iyi modeli getir"""
        if not self.trained_models:
            return None
        
        best_model = None
        best_score = float('inf') if metric == 'mae' else 0
        
        for name, info in self.trained_models.items():
            if 'performance' in info and metric in info['performance']:
                score = info['performance'][metric]
                if metric == 'mae' and score < best_score:
                    best_score = score
                    best_model = name
                elif metric == 'accuracy' and score > best_score:
                    best_score = score
                    best_model = name
        
        return best_model

# Global model registry
model_registry = ModelRegistry()

# Model tanımları
AVAILABLE_MODELS = {
    'N-Beats': {
        'description': 'Neural Basis Expansion Analysis - Interpretable time series forecasting',
        'requires': 'PyTorch',
        'features': ['Interpretable', 'Fast training', 'Trend analysis'],
        'default_params': {
            'sequence_length': 100,
            'hidden_size': 256,
            'num_stacks': 3,
            'num_blocks': 2,
            'learning_rate': 0.001,
            'epochs': 50,
            'batch_size': 32
        }
    },
    'TFT': {
        'description': 'Temporal Fusion Transformer - Multi-horizon forecasting with attention',
        'requires': 'PyTorch',
        'features': ['Attention mechanism', 'Multi-horizon', 'Feature importance'],
        'default_params': {
            'sequence_length': 150,
            'hidden_size': 128,
            'num_heads': 4,
            'num_layers': 2,
            'learning_rate': 0.001,
            'epochs': 40,
            'batch_size': 16
        }
    },
    'LSTM': {
        'description': 'Modern LSTM with Attention - Sequential learning with multi-output',
        'requires': 'TensorFlow',
        'features': ['Sequential memory', 'Multi-output', 'Bidirectional'],
        'default_params': {
            'sequence_length': 100,
            'lstm_units': 128,
            'dropout_rate': 0.2,
            'learning_rate': 0.001,
            'epochs': 50,
            'batch_size': 32
        }
    },
    'Ensemble': {
        'description': 'Ensemble of multiple models - Combines predictions from different models',
        'requires': 'All',
        'features': ['Best performance', 'Robust predictions', 'Confidence estimation'],
        'default_params': {
            'models': ['N-Beats', 'TFT', 'LSTM'],
            'weights': [0.33, 0.33, 0.34],
            'method': 'weighted_average'
        }
    }
}

print("🏗️ Model Registry hazır!")
print(f"📊 Mevcut model türleri: {list(AVAILABLE_MODELS.keys())}")

In [ ]:
# 🎮 Interactive Training Interface
class JetXTrainingInterface:
    def __init__(self, model_registry):
        self.model_registry = model_registry
        self.current_training = None
        self.setup_widgets()
        
    def setup_widgets(self):
        """Widget'ları kurulum"""
        
        # Model seçimi
        self.model_selector = widgets.Dropdown(
            options=list(AVAILABLE_MODELS.keys()),
            value='N-Beats',
            description='Model:',
            style={'description_width': 'initial'}
        )
        
        # Parametre widget'ları
        self.sequence_length = widgets.IntSlider(
            value=100, min=50, max=200, step=10,
            description='Sequence Length:',
            style={'description_width': 'initial'}
        )
        
        self.epochs = widgets.IntSlider(
            value=50, min=10, max=100, step=10,
            description='Epochs:',
            style={'description_width': 'initial'}
        )
        
        self.batch_size = widgets.Dropdown(
            options=[16, 32, 64, 128],
            value=32,
            description='Batch Size:',
            style={'description_width': 'initial'}
        )
        
        self.learning_rate = widgets.FloatLogSlider(
            value=0.001, base=10, min=-5, max=-1,
            description='Learning Rate:',
            style={'description_width': 'initial'}
        )
        
        # Eğitim kontrol butonları
        self.train_button = widgets.Button(
            description='🚀 Eğitimi Başlat',
            button_style='success',
            layout=widgets.Layout(width='200px')
        )
        
        self.stop_button = widgets.Button(
            description='⏹️ Durdur',
            button_style='danger',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        
        # Progress bar
        self.progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description='Progress:',
            bar_style='info',
            layout=widgets.Layout(width='500px')
        )
        
        # Output alanları
        self.output_area = widgets.Output()
        self.model_info_area = widgets.Output()
        
        # Event handler'lar
        self.model_selector.observe(self.on_model_change, names='value')
        self.train_button.on_click(self.on_train_click)
        self.stop_button.on_click(self.on_stop_click)
        
        # İlk model bilgisini göster
        self.update_model_info()
        
    def on_model_change(self, change):
        """Model değiştiğinde parametreleri güncelle"""
        model_name = change['new']
        if model_name in AVAILABLE_MODELS:
            params = AVAILABLE_MODELS[model_name]['default_params']
            
            # Parametreleri güncelle
            if 'sequence_length' in params:
                self.sequence_length.value = params['sequence_length']
            if 'epochs' in params:
                self.epochs.value = params['epochs']
            if 'batch_size' in params:
                self.batch_size.value = params['batch_size']
            if 'learning_rate' in params:
                self.learning_rate.value = params['learning_rate']
        
        self.update_model_info()
    
    def update_model_info(self):
        """Model bilgisini güncelle"""
        with self.model_info_area:
            clear_output()
            model_name = self.model_selector.value
            if model_name in AVAILABLE_MODELS:
                info = AVAILABLE_MODELS[model_name]
                print(f"📊 {model_name} Model Bilgisi:")
                print(f"📝 {info['description']}")
                print(f"🔧 Gereksinimler: {info['requires']}")
                print(f"✨ Özellikler: {', '.join(info['features'])}")
    
    def on_train_click(self, button):
        """Eğitim butonuna tıklandığında"""
        self.start_training()
    
    def on_stop_click(self, button):
        """Durdur butonuna tıklandığında"""
        self.stop_training()
    
    def start_training(self):
        """Eğitimi başlat"""
        self.train_button.disabled = True
        self.stop_button.disabled = False
        self.progress.value = 0
        self.current_training = True
        
        # Eğitim parametrelerini al
        config = {
            'model_type': self.model_selector.value,
            'sequence_length': self.sequence_length.value,
            'epochs': self.epochs.value,
            'batch_size': self.batch_size.value,
            'learning_rate': self.learning_rate.value
        }
        
        with self.output_area:
            clear_output()
            print(f"🚀 {config['model_type']} eğitimi başlatılıyor...")
            print(f"📊 Parametreler: {config}")
            
            # Model eğitimini başlat
            self.train_model(config)
    
    def stop_training(self):
        """Eğitimi durdur"""
        self.train_button.disabled = False
        self.stop_button.disabled = True
        self.current_training = None
        
        with self.output_area:
            print("⏹️ Eğitim durduruldu!")
    
    def train_model(self, config):
        """Model eğitimi (simülasyon)"""
        model_type = config['model_type']
        epochs = config['epochs']
        
        try:
            # Eğitim simülasyonu
            for epoch in range(epochs):
                if self.current_training is None:  # Durduruldu
                    break
                
                # Progress güncelle
                progress_percent = int((epoch + 1) / epochs * 100)
                self.progress.value = progress_percent
                
                # Simüle edilmiş loss değerleri
                train_loss = np.random.uniform(0.1, 1.0) * np.exp(-epoch * 0.05)
                val_loss = train_loss * np.random.uniform(1.0, 1.2)
                
                with self.output_area:
                    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
                
                time.sleep(0.1)  # Simülasyon için bekleme
            
            # Eğitim tamamlandı
            if self.current_training is not None:
                self.complete_training(config)
            
        except Exception as e:
            with self.output_area:
                print(f"❌ Eğitim hatası: {e}")
        finally:
            self.train_button.disabled = False
            self.stop_button.disabled = True
    
    def complete_training(self, config):
        """Eğitimi tamamla"""
        # Simüle edilmiş performans metrikleri
        performance = {
            'mae': np.random.uniform(0.1, 0.5),
            'rmse': np.random.uniform(0.2, 0.8),
            'accuracy': np.random.uniform(0.7, 0.95),
            'f1_score': np.random.uniform(0.6, 0.9)
        }
        
        # Model'i kayıt et
        model_name = f"{config['model_type']}_{datetime.now().strftime('%H%M%S')}"
        self.model_registry.register_model(
            model_name, config['model_type'], config, performance
        )
        
        with self.output_area:
            print(f"\n✅ {config['model_type']} eğitimi tamamlandı!")
            print(f"📊 Performans:")
            for metric, value in performance.items():
                print(f"   - {metric.upper()}: {value:.4f}")
            print(f"💾 Model kaydedildi: {model_name}")
        
        self.progress.value = 100
        self.current_training = None
    
    def display_interface(self):
        """Arayüzü göster"""
        # Model seçimi ve bilgi bölümü
        model_section = widgets.VBox([
            widgets.HTML("<h3>🎯 Model Seçimi</h3>"),
            self.model_selector,
            self.model_info_area
        ])
        
        # Parametre ayarları
        params_section = widgets.VBox([
            widgets.HTML("<h3>⚙️ Parametre Ayarları</h3>"),
            self.sequence_length,
            self.epochs,
            self.batch_size,
            self.learning_rate
        ])
        
        # Eğitim kontrolleri
        training_section = widgets.VBox([
            widgets.HTML("<h3>🚀 Eğitim Kontrolleri</h3>"),
            widgets.HBox([self.train_button, self.stop_button]),
            self.progress
        ])
        
        # Ana arayüz
        main_interface = widgets.VBox([
            widgets.HTML("<h2>🎮 JetX Model Trainer Interface</h2>"),
            widgets.HBox([model_section, params_section]),
            training_section,
            widgets.HTML("<h3>📊 Eğitim Çıktısı</h3>"),
            self.output_area
        ])
        
        display(main_interface)


In [ ]:
# 🎮 Ana Arayüzü Göster
training_interface = JetXTrainingInterface(model_registry)
print("🎮 Training Interface hazır!")
training_interface.display_interface()

In [ ]:
# 📊 Model Karşılaştırma Dashboard'u
class ModelComparisonDashboard:
    def __init__(self, model_registry):
        self.model_registry = model_registry
        self.setup_widgets()
    
    def setup_widgets(self):
        """Widget'ları kurulum"""
        self.refresh_button = widgets.Button(
            description='🔄 Yenile',
            button_style='info',
            layout=widgets.Layout(width='150px')
        )
        
        self.comparison_output = widgets.Output()
        
        self.refresh_button.on_click(self.on_refresh_click)
    
    def on_refresh_click(self, button):
        """Yenile butonuna tıklandığında"""
        self.update_comparison()
    
    def update_comparison(self):
        """Model karşılaştırmasını güncelle"""
        with self.comparison_output:
            clear_output()
            
            if not self.model_registry.trained_models:
                print("⚠️ Henüz eğitilmiş model yok. Önce model eğitin!")
                return
            
            # Model özeti
            print(self.model_registry.get_model_summary())
            
            # En iyi model
            best_mae = self.model_registry.get_best_model('mae')
            best_acc = self.model_registry.get_best_model('accuracy')
            
            if best_mae:
                print(f"🏆 En İyi Model (MAE): {best_mae}")
            if best_acc:
                print(f"🏆 En İyi Model (Accuracy): {best_acc}")
            
            # Performans grafiği
            self.plot_performance_comparison()
    
    def plot_performance_comparison(self):
        """Performans karşılaştırma grafiği"""
        models = list(self.model_registry.trained_models.keys())
        if len(models) < 2:
            print("📊 Karşılaştırma için en az 2 model gerekli")
            return
        
        # Metrik verilerini hazırla
        metrics = ['mae', 'rmse', 'accuracy', 'f1_score']
        data = {metric: [] for metric in metrics}
        
        for model_name in models:
            model_info = self.model_registry.trained_models[model_name]
            perf = model_info.get('performance', {})
            
            for metric in metrics:
                data[metric].append(perf.get(metric, 0))
        
        # Plotly ile görselleştirme
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('MAE (Lower is Better)', 'RMSE (Lower is Better)', 
                          'Accuracy (Higher is Better)', 'F1-Score (Higher is Better)'))
        
        # MAE
        fig.add_trace(
            go.Bar(x=models, y=data['mae'], name='MAE', marker_color='red'),
            row=1, col=1
        )
        
        # RMSE
        fig.add_trace(
            go.Bar(x=models, y=data['rmse'], name='RMSE', marker_color='orange'),
            row=1, col=2
        )
        
        # Accuracy
        fig.add_trace(
            go.Bar(x=models, y=data['accuracy'], name='Accuracy', marker_color='green'),
            row=2, col=1
        )
        
        # F1-Score
        fig.add_trace(
            go.Bar(x=models, y=data['f1_score'], name='F1-Score', marker_color='blue'),
            row=2, col=2
        )
        
        fig.update_layout(
            title_text="📊 Model Performans Karşılaştırması",
            showlegend=False,
            height=600
        )
        
        fig.show()
    
    def display_dashboard(self):
        """Dashboard'u göster"""
        dashboard = widgets.VBox([
            widgets.HTML("<h2>📊 Model Karşılaştırma Dashboard'u</h2>"),
            self.refresh_button,
            self.comparison_output
        ])
        
        display(dashboard)
        
        # İlk yükleme
        self.update_comparison()


In [ ]:
# 📊 Model Karşılaştırma Dashboard'unu Göster
comparison_dashboard = ModelComparisonDashboard(model_registry)
print("📊 Model Comparison Dashboard hazır!")
comparison_dashboard.display_dashboard()

# 📚 Kullanım Kılavuzu

## 🎯 Model Seçimi

### N-Beats
- **En İyi**: Trend analizi, hızlı eğitim
- **Önerilen**: Başlangıç için ideal
- **Parametreler**: Sequence length 50-200, epochs 30-100

### TFT (Temporal Fusion Transformer)
- **En İyi**: Karmaşık pattern'lar, attention analizi
- **Önerilen**: Yüksek performans istiyorsanız
- **Parametreler**: Sequence length 100-200, epochs 20-60

### LSTM
- **En İyi**: Sequential pattern'lar, çoklu çıktı
- **Önerilen**: Balanced performance
- **Parametreler**: Sequence length 50-150, epochs 30-80

### Ensemble
- **En İyi**: Maksimum performans
- **Önerilen**: Diğer modeller eğitildikten sonra
- **Gereksinim**: En az 2 model eğitilmiş olmalı

## ⚙️ Parametre Optimizasyonu

### Sequence Length
- **50-100**: Hızlı eğitim, temel pattern'lar
- **100-150**: Balanced performance
- **150-200**: Karmaşık pattern'lar, daha uzun eğitim

### Epochs
- **10-30**: Hızlı test
- **30-60**: Normal eğitim
- **60-100**: Maksimum performans

### Batch Size
- **16**: Daha az GPU memory, daha stabil
- **32**: Balanced (önerilen)
- **64-128**: Daha hızlı eğitim, daha fazla memory

### Learning Rate
- **0.0001**: Konservatif, stabil
- **0.001**: Önerilen başlangıç
- **0.01**: Agresif, hızlı öğrenme

## 🚀 Eğitim İpuçları

1. **GPU Kontrolü**: Runtime > Change runtime type > GPU
2. **Veri Hazırlığı**: Önce veri hazırlama hücrelerini çalıştırın
3. **Parametre Tuning**: Küçük dataset'te hızlı test yapın
4. **Model Karşılaştırma**: Birden fazla model eğitin
5. **Kaydetme**: Modeller otomatik kaydedilir

## 📊 Performans Metrikleri

- **MAE**: Mean Absolute Error (düşük daha iyi)
- **RMSE**: Root Mean Square Error (düşük daha iyi)
- **Accuracy**: Classification accuracy (yüksek daha iyi)
- **F1-Score**: Balanced precision/recall (yüksek daha iyi)

---

**🎉 Başarılı eğitimler!**

# 🔄 Gerçek Rolling Window Training Sistemi

Aşağıdaki hücreler **gerçek model eğitimi** için rolling window training sistemini içerir. Bu sistem:

## 🎯 **Rolling Window Yaklaşımı**
- 📊 **Chunk 1**: 1-1000 verisi ile eğitim → 1001-2000 verisi ile test
- 📊 **Chunk 2**: 1-2000 verisi ile eğitim → 2001-3000 verisi ile test
- 📊 **Chunk 3**: 1-3000 verisi ile eğitim → 3001-4000 verisi ile test
- 📊 **...**

## 🚀 **Gerçek Model Eğitimi**
- N-Beats, TFT, LSTM modellerini **gerçekten** eğitir
- Gerçek loss değerleri ve performans metrikleri
- GPU ile hızlı eğitim
- Modelleri .pth dosyası olarak kaydeder

## 📈 **Cycle-by-Cycle Analiz**
- Her cycle'da performans iyileşmesi
- En iyi cycle'ı otomatik bulur
- Evolution grafikleri

**⚠️ Bu gerçek eğitim sistemi daha uzun sürer (15-30 dakika) ama gerçek sonuçlar verir!**

---

In [ ]:
# 🔄 Rolling Window Training Integration Import

# Import rolling integration functions
exec(open('/content/predictor_1/colab/rolling_integration.py').read())

# Import rolling training system
exec(open('/content/predictor_1/colab/rolling_training_system.py').read())

print("✅ Rolling Window Training System yüklendi!")
print("📊 Hazır sistemler:")
print("   - RollingWindowTrainer: Gerçek model eğitimi")
print("   - RealJetXTrainingInterface: Interactive arayüz")
print("   - download_trained_models(): Model indirme")
print("   - analyze_training_results(): Sonuç analizi")

In [ ]:
# 🚀 Gerçek Rolling Window Training Interface

# Create real rolling trainer
try:
    if 'ROLLING_CHUNKS' in globals() and 'model_registry' in globals():
        real_trainer = RealJetXTrainingInterface(model_registry, ROLLING_CHUNKS)
        print("✅ Gerçek Rolling Window Trainer hazır!")
        print(f"📊 Chunk sayısı: {len(ROLLING_CHUNKS)}")
        print(f"📊 Her chunk boyutu: ~1000 kayıt")
        print(f"📊 Toplam veri: {sum(len(chunk) for chunk in ROLLING_CHUNKS)} kayıt")
        print("\n🎮 Arayüzü göstermek için aşağıdaki komutu çalıştırın:")
        print("real_trainer.display_interface()")
        
        # Interface'i göster
        real_trainer.display_interface()
        
    else:
        print("❌ ROLLING_CHUNKS veya model_registry bulunamadı!")
        print("🔧 Önce yukarıdaki veri hazırlama hücrelerini çalıştırın.")
        
except Exception as e:
    print(f"❌ Real trainer kurulum hatası: {e}")
    print("🔧 Tüm hücreleri sırayla çalıştırdığınızdan emin olun.")

In [ ]:
# 📥 Model Download Interface

print("📥 Model İndirme Arayüzü")
print("Eğitilmiş modelleri bilgisayarınıza indirin.")
print("=" * 50)

# Create download interface
try:
    download_interface = create_download_interface()
    display(download_interface)
    
    # Results analysis button
    analysis_button = widgets.Button(
        description='📊 Sonuçları Analiz Et',
        button_style='warning',
        layout=widgets.Layout(width='200px')
    )
    
    def on_analysis_click(button):
        analyze_training_results()
    
    analysis_button.on_click(on_analysis_click)
    
    analysis_interface = widgets.VBox([
        widgets.HTML("<h3>📊 Sonuç Analizi</h3>"),
        widgets.HTML("<p>Rolling training sonuçlarını analiz edin.</p>"),
        analysis_button
    ])
    
    display(analysis_interface)
    
except Exception as e:
    print(f"❌ Download interface hatası: {e}")
    print("🔧 Manuel indirme için:")
    print("   download_trained_models()")
    print("   analyze_training_results()")

# 🎯 Sistem Kullanım Rehberi

## 📋 **Adım Adım Kullanım**

### 1. **Hızlı Test (Simülasyon)**
- Yukarıdaki "🎮 JetX Model Trainer Interface" kullanın
- Hızlı test için ideal
- Simülasyon sonuçları

### 2. **Gerçek Model Eğitimi**
- "🔄 Gerçek Rolling Window Training" kullanın
- Gerçek model eğitimi (15-30 dakika)
- Production-ready modeller

### 3. **Model İndirme**
- "📥 Model Download Interface" kullanın
- Eğitilmiş modelleri bilgisayarınıza indirin
- Metadata dosyaları dahil

## 🎮 **Hangi Sistemi Kullanmalıyım?**

### 🎯 **Simülasyon Sistemi**
- ✅ Hızlı test
- ✅ Arayüz öğrenme
- ✅ Parametre deneme
- ❌ Sahte sonuçlar

### 🔄 **Rolling Window Sistemi**
- ✅ Gerçek model eğitimi
- ✅ Production-ready
- ✅ Cycle-by-cycle analiz
- ✅ Bilgisayarınızda kullanılabilir
- ❌ Daha uzun sürer

## 🚀 **Önerilen Workflow**

1. **Simülasyon ile test edin** (parametreleri öğrenin)
2. **Rolling window ile gerçek eğitim yapın**
3. **En iyi modeli indirin**
4. **Bilgisayarınızda kullanın**

---

**🎉 Artık hem simülasyon hem de gerçek model eğitimi yapabilirsiniz!**